<a href="https://colab.research.google.com/github/ykenkou001/diveintocode-ml/blob/master/week4_session1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/My Drive/Colab Notebooks/week4_session1

/content/drive/My Drive/Colab Notebooks/week4_session1


In [0]:

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# week4 授業課題 信用情報の学習
# 【問題1】コンペティション内容の確認
### コンペティションのOverviewページ読み、「Home Credit Default Risk」について以下の観点について確認してください。  
### 何を学習し、何を予測するのか どのようなファイルを作りKaggleに提出するか 提出されたものはどういった指標値で評価されるのか

### 122個ある特徴量からその人が返済する確率を予測する
IDとTARGETがカラムとして入っているcsvファイルを提出する
TARGETにはその人が返済する確率が入っている


提出された課題はAUCで評価される

In [4]:
pwd

'/content/drive/My Drive/Colab Notebooks/week4_session1'

In [5]:
!unzip Data.zip -d week4_session1

Archive:  Data.zip
replace week4_session1/Data/application_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
test_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/week4_session1/Data/application_test.csv")
train_df=pd.read_csv("/content/drive/My Drive/Colab Notebooks/week4_session1/Data/application_train.csv")

In [6]:
display(train_df.head())
display(test_df.head())

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,...,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,...,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,...,NaN,0.0514,NaN,NaN,NaN,block of flats,0.0392,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,Drivers,2.0,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-856.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.026392,-13976,-1866,-2000.0,-4208,NaN,1,1,0,1,1,0,Sales staff,4.0,2,2,WEDNESDAY,11,0,0,0,0,0,0,Business Entity Type 3,...,0.2446,0.3739,0.0388,0.0817,reg oper account,block of flats,0.3700,Panel,No,0.0,0.0,0.0,0.0,-1805.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.010032,-13040,-2191,-4000.0,-4262,16.0,1,1,1,1,0,0,NaN,3.0,2,2,FRIDAY,5,0,0,0,0,1,1,Business Entity Type 3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-821.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#欠損値がある列を削除
train_drop_df = train_df.dropna(axis=1)
test_drop_df = test_df.dropna(axis=1)

In [0]:
#objectタイプの列を削除する
object_columns = train_drop_df.select_dtypes(include=object).columns
train_drop_df = train_drop_df.drop(object_columns, axis=1)
test_drop_df = test_drop_df.drop(object_columns, axis=1)

In [0]:
X = train_drop_df.drop("TARGET", axis=1)
y = train_drop_df["TARGET"]

In [0]:
# X_testの列をXに合わせる
X_test = test_drop_df[X.columns]

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

In [12]:
forest = RandomForestClassifier(n_estimators=10)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_val_pred = forest.predict(X_val)

In [14]:
def evaluate(y_test, y_pred):
    """
    2値分類の評価指標を計算する
    """
    acc = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    confusion = metrics.confusion_matrix(y_test, y_pred)
    return acc, precision, recall, f1, confusion

print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_val, y_val_pred)))

acc : 0.9199771065844585
precision : 0.11392405063291139
recall : 0.0014775898867181087
f1 : 0.0029173419773095626
confusion matrix 
 [[70717    70]
 [ 6082     9]]


# 【問題3】テストデータに対する推定
テストデータ（application_test.csv）に対して推定を行い、Kaggleに提出を行ってください。

正しく提出が行えていれば、精度は低くても構いません。

In [0]:
y_pred = forest.predict(X_test)

# 4.特徴量エンジニアリング
ベースラインモデルを元に、入力する特徴量に様々な工夫を行い精度を向上させていきます。

# 【問題4】特徴量エンジニアリング¶
精度を上げるために以下のような観点で 特徴量エンジニアリング（Feature Engineering） を行ってください。  
・どの特徴量を使うか  
・どう前処理をするか  
何をした時に検証データに対する評価指標がどのようになったかをまとめてください。最低5パターンの学習・検証を行ってください。  

精度が高かったものに関してはテストデータに対しても推定を行い、Kaggleに提出を行ってください。  

## ・欠損値を０で埋める。

In [16]:
train_drop_df = train_df.fillna(0)
test_drop_df = test_df.fillna(0)

#objectタイプの列を削除する
object_columns = train_drop_df.select_dtypes(include=object).columns
train_drop_df = train_drop_df.drop(object_columns, axis=1)
test_drop_df = test_drop_df.drop(object_columns, axis=1)

X = train_drop_df.drop("TARGET", axis=1)
y = train_drop_df["TARGET"]

# X_testの列をXに合わせる
X_test = test_drop_df[X.columns]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

forest.fit(X_train, y_train)
y_val_pred = forest.predict(X_val)

print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_val, y_val_pred)))

acc : 0.920224251411327
precision : 0.3793103448275862
recall : 0.01083565916926613
f1 : 0.02106943335993615
confusion matrix 
 [[70679   108]
 [ 6025    66]]


## ・欠損値を平均値で埋める。

In [0]:

train_drop_df = train_df.fillna(train_df.mean())
test_drop_df = test_df.fillna(test_df.mean())

#objectタイプの列を削除する
object_columns = train_drop_df.select_dtypes(include=object).columns
train_drop_df = train_drop_df.drop(object_columns, axis=1)
test_drop_df = test_drop_df.drop(object_columns, axis=1)

X = train_drop_df.drop("TARGET", axis=1)
y = train_drop_df["TARGET"]

# X_testの列をXに合わせる
X_test = test_drop_df[X.columns]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

forest.fit(X_train, y_train)
y_val_pred = forest.predict(X_val)

print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_val, y_val_pred)))

## ・欠損値を中央値で埋める。

In [17]:
train_drop_df = train_df.fillna(train_df.median())
test_drop_df = test_df.fillna(test_df.median())

#objectタイプの列を削除する
object_columns = train_drop_df.select_dtypes(include=object).columns
train_drop_df = train_drop_df.drop(object_columns, axis=1)
test_drop_df = test_drop_df.drop(object_columns, axis=1)

X = train_drop_df.drop("TARGET", axis=1)
y = train_drop_df["TARGET"]

# X_testの列をXに合わせる
X_test = test_drop_df[X.columns]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

forest.fit(X_train, y_train)
y_val_pred = forest.predict(X_val)

print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_val, y_val_pred)))

acc : 0.9194437940633211
precision : 0.27631578947368424
recall : 0.010343129207026762
f1 : 0.019939863902516224
confusion matrix 
 [[70622   165]
 [ 6028    63]]


## ・欠損値を最頻値で埋める。

In [0]:
train_drop_df = train_df.fillna(train_df.mode())
test_drop_df = test_df.fillna(test_df.mode())

#objectタイプの列を削除する
object_columns = train_drop_df.select_dtypes(include=object).columns
train_drop_df = train_drop_df.drop(object_columns, axis=1)
test_drop_df = test_drop_df.drop(object_columns, axis=1)

X = train_drop_df.drop("TARGET", axis=1)
y = train_drop_df["TARGET"]

# X_testの列をXに合わせる
X_test = test_drop_df[X.columns]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

forest.fit(X_train, y_train)
y_val_pred = forest.predict(X_val)

print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_val, y_val_pred)))

## ・重要そうな特徴量を選んで残りは削除する。

In [19]:
#objectタイプの列を削除する
select_columns = ["AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY"]
train_drop_df = train_df[select_columns]
test_drop_df = test_df[select_columns]

train_drop_df = train_drop_df.dropna(axis=1)
test_drop_df = test_drop_df.dropna(axis=1)

X = train_drop_df
y = train_df["TARGET"]

# X_testの列をXに合わせる
X_test = test_drop_df
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

forest.fit(X_train, y_train)
y_val_pred = forest.predict(X_val)

print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_val, y_val_pred)))

acc : 0.9102994354691849
precision : 0.08116545265348596
recall : 0.012805779018223608
f1 : 0.022121384004537722
confusion matrix 
 [[69904   883]
 [ 6013    78]]
